In [77]:
import pandas as pd
import sys
import sqlalchemy
import os


%matplotlib inline

CONN = sqlalchemy.create_engine('sqlite:///data.db')


#CONN.execute('PRAGMA table_info(player_stats)').fetchall()
#CONN.execute(sqlalchemy.text('SELECT player_id, full_name from player_stats where player_id = :p'),p=101106).fetchone()


In [78]:
os.listdir('../Sports')

['.DS_Store', 'Baseball', 'Basketball', 'E-Sports', 'Football', 'Soccer']

In [79]:
nba_data_path = '../Sports/Basketball/Nba Data.csv'
players_path = '../Sports/Basketball/Players.csv'
season_stats = '../Sports/Basketball/Seasons_Stats.csv'

nba_df = pd.read_csv(nba_data_path)
players_df = pd.read_csv(players_path)
season_stats_df = pd.read_csv(season_stats)


player_percentages = nba_df[list([column for column in nba_df.columns if ('Percentage' in column and 'Rebound' not in column and 'Effective' not in column and 'Against' not in column) or column == 'Name'])]

player_percentages.to_sql('player_percentages',CONN,if_exists='replace')
players_df.to_sql('player',CONN, if_exists='replace')


players_df.columns = ['id','name','height','weight','college','born','birth_city','birth_state']

columns = list(season_stats_df.columns)
columns[0] = 'id'
season_stats_df.columns = columns


duplicate_players_total = nba_df[(nba_df.duplicated(subset=['Full Name'])) & (nba_df['Team'] == 'TOTAL')]

nba_df = nba_df.drop_duplicates(subset=['Full Name'], keep=False)

nba_player_stats_df = pd.concat([nba_df, duplicate_players_total])

nba_player_stats_cols = list(nba_player_stats_df.columns)
nba_player_stats_cols[32] = 'Second Pass Hockey Assist'
nba_player_stats_cols = ['_'.join(column.strip().lower().split())  for column in nba_player_stats_cols]

nba_player_stats_df.columns = nba_player_stats_cols


nba_player_stats_df.team = nba_player_stats_df.team.apply(lambda x : x.lower())


nbapstatsdfkmeans = nba_player_stats_df[['player_id','games_played',
 'minutes',
 'pull_up_shots_points_per_game',
 'pull_up_field_goals_made_per_game',
 'pull_up_field_goals_attempted_per_game',
 'pull_up_field_goal_percentage',
 'pull_up_3_point_field_goals_made_per_game',
 'pull_up_3_point_field_goals_attempted_per_game',
 'pull_up_3_point_percentage',
 'pull_up_3_point_effective_field_goal_percentage',
 'pull_up_points_total',
 'drives_per_game',
 'points_per_game_off_drives',
 'total_points_per_game_on_drives_(including_ft)',
 'field_goal_percentage_on_drives',
 'points_per_48_minutes_on_drives',
 'total_points_on_drives',
 'total_drives',
 'blocks_per_game',
 'steals_per_game',
 'field_goals_made_against_when_defending_rim',
 'field_goals_attempted_against_when_defending_rim',
 'field_goal_percentage_against_when_defending_rim',
 'total_blocks',
 'passes_per_game',
 'assists_per_game',
 'free_throws_assisted_per_game',
 'second_pass_hockey_assist',
 'assist_opportunities_per_game',
 'points_created_off_assists_per_game',
 'points_created_off_assists_per_game.1',
 'total_assists',
 'touches_per_game',
 'front_court_touches_per_game',
 'time_of_possession',
 'close_range_touches_per_game',
 'elbow_touches_per_game',
 'points_per_game',
 'points_per_touch',
 'points_per_half_court_touch',
 'total_touches',
 'total_distance_(miles)',
 'average_speed',
 'distance_per_game_(miles)',
 'distance_per_48_minutes_(miles)',
 'total_distance_on_offense_(miles)',
 'total_distance_on_defense_(miles)',
 'average_speed_offense',
 'average_speed_defense',
 'rebounds_per_game',
 'rebound_chances',
 'rebound_percentage',
 'contested_rebounds_per_game',
 'uncontested_rebounds_per_game',
 'percentage_of_rebounds_uncontested',
 'total_rebounds',
 'offensive_rebounds_per_game',
 'offensive_rebound_chances',
 'offensive_rebounds_collected_percentage',
 'offensive_rebounds_contested',
 'offensive_rebounds_uncontested',
 'offensive_rebounds_uncontested_percentage',
 'defensive_rebounds',
 'defensive_rebound_chances',
 'defensive_rebounds_collected_percentage',
 'defensive_rebounds_contested',
 'defensive_rebounds_uncontested',
 'defensive_rebounds_uncontested_percentage',
 'catch_and_shoot_points_per_game',
 'catch_and_shoot_field_goal_made_per_game',
 'catch_and_shoot_field_goal_attempted_per_game',
 'catch_and_shoot_field_goal_percentage',
 'catch_and_shoot_3_point_made_per_game',
 'catch_and_shoot_3_point_attempts_per_game',
 'catch_and_shoot_3_point_field_goal_percentage',
 'catch_and_shoot_effective_field_goal_percentage',
 'catch_and_shoot_points_total',
 'points_per_game.1',
 'points_per_game_drive',
 'field_goal_percentage_drive',
 'points_per_game_close_range',
 'field_goal_percentage_close_range',
 'points_per_game_catch_and_shoot',
 'field_goal_percentage_catch_and_shoot',
 'points_per_game_pull_up',
 'field_goal_percentage_pull_up',
 'field_goal_attempted_drive',
 'field_goal_attempted_close_range',
 'field_goal_attempted_catch_and_shoot',
 'field_goal_attempted_pull_up',
 'effective_field_goal_percentage']]




In [80]:
# Solving null values problem in df. 
nbapstatsdfkmeans =  nbapstatsdfkmeans.fillna(value=0.0)






In [101]:
from sklearn.cluster import KMeans
import collections

kmeans = KMeans().fit(nbapstatsdfkmeans)

kmeans.labels_


clustering = collections.defaultdict(list)

for idx, label in enumerate(kmeans.labels_):
    clustering[label].append(idx)
 

name_lookup = {record['player_id']:record['full_name'] for record in nba_player_stats_df[['player_id','full_name']].to_dict('records')}


for cluster in clustering:
    print("Cluster: %d"%(cluster))
    player_cluster = nbapstatsdfkmeans[nbapstatsdfkmeans.index.isin(clustering[cluster])][['player_id']]
    player_cluster.name = player_cluster.player_id.apply(lambda x : name_lookup[x])
    print(player_cluster.name)
    


Cluster: 0
47             Al Harrington
50               Elton Brand
56         Metta World Peace
57          Andrei Kirilenko
58             Manu Ginobili
59           Andrea Bargnani
65                 JJ Redick
67           Thabo Sefolosha
71           Shawne Williams
77             Jordan Farmar
78             Joel Freeland
81                 PJ Tucker
82              Paul Millsap
87           Jose Juan Barea
88               Mike Harris
89                 Greg Oden
90              Kevin Durant
94              Corey Brewer
96               Joakim Noah
101           Rodney Stuckey
103          Marco Belinelli
104              Jason Smith
110            Arron Afflalo
111           Tiago Splitter
117               Marc Gasol
129         Anthony Tolliver
130          Michael Beasley
139              Brook Lopez
143           Jason Thompson
146              Robin Lopez
149             JaVale McGee
151            Alexis Ajinca
161                 DJ White
169    Chris Douglas-Roberts
170